# 3.3 线性回归的简洁实现

In [48]:
import torch
from torch import nn
import numpy as np
# torch.manual_seed(1) # 没啥用

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.8.1


## 3.3.1 生成数据集

In [49]:
# 生成数据与从零实现一致
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 3.3.2 读取数据

In [50]:
# 读取数据与从零实现不同，引入了 torch.utils.data, 并调用 TensorDataset 方法将训练特征与标签存在一起，
# 最后调用 DataLoader 生成指定步长的数据迭代器

import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [76]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[-1.5114,  0.8042],
        [ 1.4863,  1.0607],
        [ 0.1773,  0.0867],
        [ 0.2400,  0.6184],
        [ 1.6775, -0.3517],
        [ 0.8721,  0.9203],
        [ 0.7155, -1.5766],
        [-0.5613,  0.4703],
        [-0.5623,  0.2523],
        [-0.9451, -0.2246]]) 
 tensor([-1.5455,  3.5759,  4.2594,  2.5704,  8.7586,  2.8100, 10.9988,  1.4704,
         2.2136,  3.0835])


## 3.3.3 定义模型

In [79]:
# 使用 nn.Sequential 定义线性模型， 使用 nn.Linear 添加线性层，与从零配置不同，不需要显示定义

# 写法一， nn.Sequential 定义连续模型
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

print(net[0])
print(net[0].weight)
print(net[0].bias)


Linear(in_features=2, out_features=1, bias=True)
Parameter containing:
tensor([[-0.6201,  0.4772]], requires_grad=True)
Parameter containing:
tensor([-0.6368], requires_grad=True)


In [80]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.6201,  0.4772]], requires_grad=True)
Parameter containing:
tensor([-0.6368], requires_grad=True)


## 3.3.4 初始化模型参数

In [81]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [82]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0121,  0.0021]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [83]:
loss = nn.MSELoss()


<generator object Module.parameters at 0x7fd7efcd80a0>


## 3.3.6 定义优化算法

In [84]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [12]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [13]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [88]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        # 自动求导
        l.backward()
        # 下一轮迭代优化
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000234
epoch 2, loss: 0.000197
epoch 3, loss: 0.000103


In [75]:
# 取神经网络第一层作为全链接层
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 2.0002, -3.3998]])
4.2 tensor([4.1998])
